In [ ]:
from keras.datasets import mnist
import keras
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image as im
import os
import time

In [ ]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

In [ ]:
def showImage(arr):
    plt.imshow(arr, cmap = 'gray')
    plt.show()

In [ ]:
train_images = train_images.reshape((60000,28,28,1))
train_images = train_images.astype('float32')/255

test_images = test_images.reshape((10000, 28,28,1))
test_images = test_images.astype('float32')/255

from keras.utils import to_categorical

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

In [ ]:
from keras import layers
from keras import models
from keras.layers import (
    BatchNormalization, SeparableConv2D, MaxPooling2D, Activation, Flatten, Dropout, Dense
)
from keras.optimizers import RMSprop, Adam

In [ ]:
from keras.models import Sequential
from keras.layers import (Dense, Conv2D, Flatten, Dropout, LeakyReLU, MaxPooling2D, Conv2DTranspose, Reshape)

def def_discriminator(in_shape=(28,28,1)):

    model = Sequential()
    model.add(Flatten())
    model.add(Dense(1024, activation=LeakyReLU(alpha=0.2)))
    model.add(Dropout(0.4))
    model.add(Dense(512, activation=LeakyReLU(alpha=0.2)))
    model.add(Dropout(0.4))
    model.add(Dense(512, activation=LeakyReLU(alpha=0.2)))
    model.add(Dense(1, activation='sigmoid'))

    opt = Adam(learning_rate=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

In [ ]:

def def_generator(latent_dim):
    model = Sequential()
    model.add(Dense(256, input_dim=latent_dim))   
    model.add(Dense(512, activation=LeakyReLU(alpha=0.2)))
    model.add(Dense(1024, activation=LeakyReLU(alpha=0.2)))
    model.add(Dense(784, activation=LeakyReLU(alpha=0.2)))
    model.add(Reshape((28,28,1)))

    return model

# def def_generator(latent_dim):
#     # Define the input layer
#     input_layer = keras.layers.Input(shape=(latent_dim,))
    
#     # Project and reshape the input
#     dense_layer = Dense(7 * 7 * 128)(input_layer)
#     # dense_layer = Reshape((7, 7, 128))(dense_layer)
    
#     # # Upsampling blocks
#     # conv_transpose1 = Conv2DTranspose(64, (3, 3), padding='same', activation=LeakyReLU(alpha=0.2))(dense_layer)
#     # conv_transpose1 = BatchNormalization()(conv_transpose1)
    
#     # conv_transpose2 = Conv2DTranspose(32, (3, 3), padding='same', activation=LeakyReLU(alpha=0.2))(conv_transpose1)
#     # conv_transpose2 = BatchNormalization()(conv_transpose2)
    

#     x = Dense(784, activation=LeakyReLU(alpha=0.1))(dense_layer)
#     x = Dense(784, activation=LeakyReLU(alpha=0.1))(x)
#     x = Dense(784, activation=LeakyReLU(alpha=0.1))(x)
#     x = Reshape((28,28,1))(x)
    
#     # Define the model
#     model = keras.Model(inputs=input_layer, outputs=x)
#     return model
generator = def_generator(100)
generator.summary()

In [ ]:
# def define_gan(generator, d):
#     d.trainable = False
#     model = Sequential()
#     model.add(generator)
#     model.add(d)
#     opt = Adam(learning_rate=0.0002, beta_1=0.5)
#     model.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer=opt)
#     return model
from keras import backend as K
def generator_loss(y_true, y_pred, generated_data):
    # Calculate mean squared error between batches
    similarity_penalty = tf.keras.backend.mean(K.square(K.mean(generated_data, axis=0) - generated_data), axis=None)
    
    return K.binary_crossentropy(y_true, y_pred) + similarity_penalty
def define_gan(generator, discriminator):
    discriminator.trainable = False
    model = Sequential()
    model.add(generator)
    model.add(discriminator)
    opt = Adam(learning_rate=0.0002, beta_1=0.5)
    model.compile(loss=lambda y_true, y_pred: generator_loss(y_true, y_pred, generator.output), metrics=['accuracy'], optimizer=opt)
    return model

In [ ]:
LATENT_DIM = 100
D_BATCH_SIZE = 10000
G_BATCH_SIZE = 256
EPOCHS = 10
DATA_LEN = 60000

In [ ]:
def re_label(arr):
    return arr.reshape(len(arr), 1)
def shuffle(ims, labels):
    if(len(ims)!=len(labels)):
        return -1
    rand_index = np.random.permutation(len(ims))
    return ims[rand_index], labels[rand_index]

In [ ]:
from tqdm import tqdm

In [ ]:
discriminator = def_discriminator()
generator = def_generator(LATENT_DIM)
gan = define_gan(generator, discriminator)
losses = []

In [ ]:
g_latent_dims = np.random.randn(500, LATENT_DIM)
gan.fit(g_latent_dims, re_label(np.ones(500)), batch_size=32)

In [ ]:
def plot_ims():
    fig, axes = plt.subplots(5, 5, figsize=(10, 10))  # Create a 5x5 subplot grid
    for i in range(5):
        for j in range(5):
            generated_image = generator.predict(np.random.randn(1, LATENT_DIM)*2, verbose=0)  # Generate an image
            axes[i, j].imshow(generated_image.reshape(28, 28), cmap='gray')  # Display the generated image
            axes[i, j].axis('off')  # Turn off axis labels
    plt.tight_layout()  # Adjust subplot layout
    plt.show()


In [ ]:
showImage(generator.predict(np.random.rand(1,100))[0,:,:,0])

In [ ]:
from keras import backend as K

In [ ]:
discriminator.trainable=True
d_latent_dims = np.random.randn(1000//2, LATENT_DIM)

rand_index = np.random.randint(0,DATA_LEN, 1000//2)

real_ims, real_labels = train_images[rand_index], re_label(np.ones(1000//2))
fake_ims, fake_labels = generator.predict(d_latent_dims, verbose=0), re_label(np.zeros(1000//2))


ims, labels = shuffle(np.vstack((real_ims, fake_ims)), np.vstack((real_labels, fake_labels)))




discriminator.fit(ims,labels, batch_size=32)


In [ ]:
g_latent_dims = np.random.randn(500, LATENT_DIM)*2
gan.fit(g_latent_dims, re_label(np.ones(500)), batch_size=32)

In [ ]:
gan.load_weights("/Users/seanyao/cs/ML Keras/GAN MNIST/gan_FNN.keras")

In [ ]:
from IPython.display import clear_output

for epoch in range(EPOCHS):
    print("EPOCH:", epoch)
    for batch in tqdm(range(DATA_LEN//G_BATCH_SIZE)):
        d_latent_dims = np.random.randn(D_BATCH_SIZE//2, LATENT_DIM)
        g_latent_dims = np.random.randn(G_BATCH_SIZE, LATENT_DIM)

        rand_index = np.random.randint(0,DATA_LEN, D_BATCH_SIZE//2)

        real_ims, real_labels = train_images[rand_index], re_label(np.ones(D_BATCH_SIZE//2))
        fake_ims, fake_labels = generator.predict(d_latent_dims, verbose=0), re_label(np.zeros(D_BATCH_SIZE//2))

        ims, labels = shuffle(np.vstack((real_ims, fake_ims)), np.vstack((real_labels, fake_labels)))

        # discriminator.fit(ims,labels, batch_size=32)
        d_loss = discriminator.train_on_batch(ims, labels)
        g_loss = gan.train_on_batch(g_latent_dims, re_label(np.ones(G_BATCH_SIZE)))

        losses.append((d_loss, g_loss))
        
        print("Discriminator Loss:", d_loss[0], "Generator Loss: ", g_loss[0])
        print("Discriminator Acc:", d_loss[1], "Generator Acc: ", g_loss[1])
        if(batch%3==0):
            clear_output()
            plot_ims()
        if(batch%10==0):
            s = "gan_FNN_more_layers.keras"
            gan.save(s)
    print("Discriminator Loss:", d_loss, " ", g_loss, "Generator Loss")
    plot_ims()




In [ ]:
s = "gan_CNN.keras"
gan.save(s)

In [ ]:
plot_ims()